In [ ]:
import numpy as np
import os
from IPython.display import display, HTML
import json
import time
import pathlib
write_embeddings = False
write_chunk_metadata = True

In [ ]:




# textpath ="/home/sasi/Desktop/Final_Sem/CS4805 BTP/Exp1/C++Safety/safety-in-context.txt"
# char_bytes = 1  we are dealing with utf-8 documents only.




##cleaning the input file
#doc variable should contain entire cleaned text.


#all invalid characters are replaced with space.
#valid characters remain the same.
def valid_char(c)->str:
    val = ord(c)
    if (val <= ord('9') and val >= ord('0')):
        return c
    if (c in ['.',',',';']):
        return '.'
    val = val|32
    if (val <= ord('z') and val >= ord('a')):
        return chr(val)

    return ' '

#function that cleans a string and returns a
# a list of words is better
def clean_doc(doc,filter = False):
    doc = list(doc)
    if filter:
        for i in range(len(doc)):
            doc[i]= valid_char(doc[i])
    
    doc = ''.join(doc)
    replace_chars = ['\n','\t','\r']
    for c in replace_chars:
        doc = doc.replace(c,' ')

    doc = doc.split(' ')
    doc = [i for i in doc if i != ""]
    return doc



# print(doc)

In [ ]:
chunk_lst = []
chunk_metadata = []
metadata_index1 = []
metadata_index2 = []

Grobid_Articles = "/home/sasi/Desktop/Final_Sem/CS4805_BTP/Exp4/Sigmetrics/researchtext/"
Wiki_Articles = "cleaned_articles/"
Wiki_Articles_metadata = "cleaned_articles_metadata/"
Wiki_Articles_embedding = "cleaned_articles_embedding/"


In [ ]:
##Chunking code
#doc variable contains entire cleaned document.
#chunk_lst variables contains the list of chunks.

#add code to clean document to remove punctuation, newlines and redundant spaces.
#also while chunking, chunk based on sentences. not length.

#ideas for embedding.
#idea-1 How about add next sentence to chunk only if current chunk embedding dot product with next sentence is above a certain threshold.
#also cap the number of sentences and words.


def chunker(chunk_lst,filename, chunk_len = 32):
  fp = open(filename,'r')
  doc = fp.read()
  fp.close()

  doc = clean_doc(doc,filter = False)

  print("Number of words in doc ", len(doc))

  chunk_cnt = 0
  while chunk_cnt < len(doc):
      le = chunk_cnt
      ri = chunk_cnt + chunk_len
      if (ri >= len(doc)):
        ri = len(doc) - 1

      while(ri < len(doc) and doc[ri][-1] != '.'):
          ri += 1
      chunk = ' '.join(doc[le: ri + 1])
      ind = int(chunk_cnt/chunk_len)
      #  print("chunk_cnt ", ind )
      #  print(chunk)
      chunk_lst.append(chunk)
      chunk_cnt += ri - le  + 1

  print("##Number of chunks",len(chunk_lst))


def valid_chunk(line):
    line =line.split(" ")

    
    if (len(line) < 16):
        return False
    
    return True
    
       
def chunker2(chunk_lst,chunk_metadata, filename):
   fp = open(filename,'r')
   offset = 0
   doc = fp.readlines()
   #get line starting offset aswell.

   fp.close()
   chunk_lst_file =[]
   chunk_metadata_file = []

   for line in doc:
      if (valid_chunk(line)):
        chunk_lst.append(line)
        
        metadata_index1.append(len(chunk_metadata))
        metadata_index2.append(len(chunk_metadata_file))
        chunk_metadata_file.append([filename,offset,len(line),len(chunk_lst) - 1])
      
      offset += len(line)

   chunk_metadata.append(chunk_metadata_file)
   print("##Number of chunks",len(chunk_lst))
   




In [ ]:

files = os.listdir(Wiki_Articles)
# files = os.listdir(Grobid_Articles)

for file in files:
    #chunker(chunk_lst, Articles_dir + file, chunk_len = 32)
    print(file)
    chunker2(chunk_lst,chunk_metadata, Wiki_Articles + file)

In [ ]:
print("Number of chunks ", len(chunk_lst))

In [ ]:
for i in range(len(chunk_lst)):
    ind1 = metadata_index1[i]
    ind2 = metadata_index2[i]

    metadata = chunk_metadata[ind1][ind2]
    print(metadata[0],metadata[1],metadata[2])

    with open(metadata[0],'r') as f:
        f.seek(metadata[1])
        print(f.read(metadata[2]))

In [ ]:
chunk_lst_len = [len(chunk_lst[i]) for i in range(len(chunk_lst))]

##get max length of chunk and its index
ind = range(len(chunk_lst))
sdata = [value for _, value in sorted(zip(chunk_lst_len, ind ),reverse=True)]

for i in range(6):
        # print(f"Chunk {i} : {chunk_corelation[sorted_data[i]]} ",  chunk_lst[sorted_data[i]])
        print(chunk_lst[sdata[i]])
        
##bar graph
import matplotlib.pyplot as plt
plt.hist(chunk_lst_len,bins = 200,range = (0,1200))
plt.savefig("Sigmetrics.png")
plt.show()


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans

# Load the Sentence Transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
def chunk_embedder_transformer(model,chunk):
     return model.encode(chunk)


def chunk_embedder(model,i,ebd_lst,save_embeddings=False):
       metadata_lst = chunk_metadata[i]

       if (len(metadata_lst) == 0):
             return
       
       filename = metadata_lst[0][0]
       filename = filename.split("/")[-1]
       filename1 = Wiki_Articles_metadata+ "/" + filename
       filename2 = Wiki_Articles_embedding+ "/" + filename
       

       #open a file-like object
       

      

       if (write_embeddings):
            with open(filename2,'wb') as fp2:
                  with open(filename1,'wb') as fp:
                        for j in range(len(metadata_lst)):
                              metadata = metadata_lst[j]
                              #convert all elements of metadata to str
                              
                              fp.write(str(metadata[1]) +"," + str(metadata[2]) +"," + str(metadata[3]) + '\n')
                              chunk =  chunk_lst[metadata[3]]
                              embedding=model.encode(chunk)
                              ebd_lst.append(embedding)
                              
                              np.savez_compressed(fp2, embedding)
                              
                              #save embedding to file
             
       else:
             for j in range(len(metadata_lst)):
                              metadata = metadata_lst[j]
                              #convert all elements of metadata to str
                              
                              # fp.write(str(metadata[1]) +"," + str(metadata[2]) +"," + str(metadata[3]) + '\n')
                              chunk =  chunk_lst[metadata[3]]
                              embedding=model.encode(chunk)
                              ebd_lst.append(embedding)
                              
                              # np.savez_compressed(fp2, embedding)
             
             





       
       

     


embed_lst = []
for i in range(len(chunk_metadata)):
#     print(i,chunk_metadata[i][0][0])
    print(i)
    chunk_embedder(model, i,embed_lst)
#     print(embed)


In [ ]:
print(chunk_metadata)

In [ ]:
#get the final text to be attended to the query
def query_corelation(query,similarity = 'cosine'):
  # print(f"The similiarity metric used is {similarity}")
  query_embed = chunk_embedder_transformer(model,' '.join(clean_doc(query)))

  chunk_corelation_dum = []

  for i in range(len(chunk_lst)):
        if similarity == 'cosine':
          chunk_corelation_dum.append(np.dot(query_embed,embed_lst[i]) / (np.linalg.norm(query_embed) * np.linalg.norm(embed_lst[i])))
        else:
          chunk_corelation_dum.append(np.dot(query_embed,embed_lst[i]))
  return chunk_corelation_dum




def query_llm(model, question):
    model_name = pathlib.Path(model.model_path).name
    time_start = time.time()
    #prompt = f"Q: {question} A:"
    prompt = question
    output = model(prompt=prompt, max_tokens=250) # if max tokens is zero, depends on n_ctx
    response = output["choices"][0]["text"]
    time_elapsed = time.time() - time_start
    display(HTML(f'<code>{model_name} response time: {time_elapsed:.02f} sec</code>'))
    display(HTML(f'<strong>Question:</strong> {question}'))
    display(HTML(f'<strong>Answer:</strong> {response}'))
    #print(json.dumps(output, indent=2))
    return response



def query_mutation(query,chunk_lst,chunk_corelation):
    k = 1
    top_k = np.partition(chunk_corelation, -k)[-k:]  # Results are not sorted
    np.sort(top_k)[::-1]
    print(top_k)
    threshold = top_k[0]
    Context = ""
    ContextHead = " Answer using the above context only. "
    
    for i in range(len(chunk_corelation)):
            if (chunk_corelation[i] >= threshold):
               print(chunk_corelation[i])
               print(chunk_lst[i])
               Context += chunk_lst[i]
               Context += " "
    
    query = Context + ContextHead + " Q:" + query + " A:"
    return query




In [ ]:
def chunking(query1,ignore_chunks,k = 3,threshold = 64):
    chunk_corelation = query_corelation(query1)


    # transformed_query = query_mutation(query1,chunk_lst,chunk_corelation)
    # # transformed_query = chunk + "Answer using the above context only. " + query1 
    # print(transformed_query)

    # print("embeddings and chunks")

    indices = list(range(len(chunk_lst)))
    #filter indices whose lenght is less than threshold
    # indices = [i for i in indices if len(chunk_lst[i]) >= threshold]
    sorted_data = [value for _, value in sorted(zip(chunk_corelation, indices ),reverse=True)]

    rel_chunk_lst = []
    Indices=[]
    cnt=0
    for i in range(len(sorted_data)):
        
        # print(f"Chunk {i} : {chunk_corelation[sorted_data[i]]} ",  chunk_lst[sorted_data[i]])
        
        j = sorted_data[i]
        if len(chunk_lst[j]) < threshold:
            continue
        if j in ignore_chunks:
            continue
        
        rel_chunk_lst.append(chunk_lst[sorted_data[i]].strip('\n'))
        Indices.append(sorted_data[i])
        cnt += 1
        # print("#In chunking : ", chunk_lst[sorted_data[i]])
        if (cnt == k):
            break
       

    # print(query1 + " Answer briefly.")

    
    rag_query=rel_chunk_lst[0] + " Answer using the above context. " + query1 + " Answer briefly."
    

    return [rag_query, rel_chunk_lst, Indices, chunk_corelation]

In [ ]:
querylist = [
    "How does heat generated on GPU affect the bandwidth offered by DRAM? Discuss about the thermal-induced NUMA due to this effect? How can the data be placed in a thermal-induced retention-aware fashion to improve performance?",
    "Why does development of large proprietary cloud providers lead to unreliability on network assessing methods such as trace-route? What other new methods can be used to assess connectivity in private backbone infrastructures.",
    "What is SLITS, a scheduler design to mitigate sparsity issue in Thread-Interaction Mix? How does it solve the sparsity issue in TIM?",
    "How does NetDiffusion address the limitations of current synthetic trace generation methods in the context of networking?",
    "What can affect the benefit due to PIM nodes, if kernel execution time scales linearly ? Explain with experimental evidence?",
    "What is MetaVRadar Tool? Why is it useful? How was its accuracy evaluated?",
    "How does GuaNuary, a safety guard against write overflows leverage Intel’s SPP to achieve less memory consumption and more efficient allocator?",
    "Why are control and data paths split in analysis in SplitRPC? How does this splitting help optimizing RPC mechanism for ML inference serving?",
    "How is FlowLiDAR capable of detecting almost all flows in the network while requiring a very reasonable amount of memory not depending on flowIDs?",
    "First telemetry tool able to fuse physical-layer channel occupancy readings from the cellular control channel with higher-layer packet arrival statistics? Explain how beneficial this is?"
]

In [ ]:
def method1retreiver(iter,initialseed, filename,query):
    
    top_k = 7

    fp = open(filename,"w")

    query_chunks= []
    # for i in range(3):
    #     print(corelation[indices[i]])
    #     print(chunk[i])
    #     print("------------")
    #     print(chunk_lst[indices[i]])
    
    
    fp.write(query + "\n")
    fp.write("------------\n\n")
    q = query
    for i in range(iter):
        lis =chunking(q,query_chunks,top_k)
        corelation=lis[3]
        indices=lis[2]
        chunk=lis[1]
        fp.write("Chunks: ")
        for j in indices:
            fp.write(str(j) +" ")
        fp.write("\n")
        for j in indices:
            fp.write(str(corelation[j]) +" ")
        fp.write("\n")
        chosenchunk = 0
        if i == 0:
            chosenchunk = initialseed
        fp.write("Choosing the top chunk "+ str(indices[chosenchunk]) + "\n")
        fp.write(chunk[chosenchunk] + "\n")
        fp.write("------------\n")
        q = chunk[chosenchunk] + "." + q
        query_chunks.append(indices[chosenchunk])



    fp.close()

    return query_chunks


In [ ]:


fp = open("dummy.txt","w")
initialseed = -1

for q in querylist:
    chunks_temp=method1retreiver(15,initialseed,"method1.txt",q)
    chunks_temp = [str(i) for i in chunks_temp]
    fp.write(" ".join(chunks_temp) + "\n")

fp.close()
    


In [ ]:
method1retreiver(15,initialseed,"method1.txt",querylist[2])

In [ ]:
import random

def method2chunkhelper(chunk_index,chunk_lst, chunk_corelation, chunk_embedding,sum,curr_chunks,top_k,unique_chunks_factor,length_factor):
    curr_delta = []
    for i in range(top_k):
        temp_sum = sum - chunk_embedding[curr_chunks[i]]
        temp_c = chunk_embedding[chunk_index] - chunk_embedding[curr_chunks[i]]
       
        temp_delta = np.dot(temp_sum,temp_c)
                
        temp_delta = (chunk_corelation[chunk_index] - chunk_corelation[curr_chunks[i]]) - (2* temp_delta *unique_chunks_factor)/(top_k - 1)    
        temp_delta += length_factor*(np.log(len(chunk_lst[curr_chunks[i]])) - np.log(len(chunk_lst[chunk_index])))
        #print log of length of chunk

        curr_delta.append(temp_delta)
     
    maxind = np.argmax(curr_delta)
    return [maxind, curr_delta[maxind]]
    

             
def method2retreiver(chunk_lst, chunk_corelation, chunk_embedding, unique_chunks_factor,length_factor,top_k):
    
    

    query_chunks= []
    #initialize query_chunks with random values
    query_chunks = [random.randint(0,len(chunk_lst) - 1) for i in range(top_k)]

    sum = None
    for i in range(top_k):
        if sum is None:
            sum = chunk_embedding[query_chunks[i]]
        else:
            sum += chunk_embedding[query_chunks[i]]
    
    # print(query_chunks)

    for iter in range(100):   
            delta_chk = []

            for i in range(len(chunk_lst)):
                if i in query_chunks:
                    continue
                temp = method2chunkhelper(i,chunk_lst, chunk_corelation, chunk_embedding, sum, query_chunks, top_k, unique_chunks_factor, length_factor)
                delta_chk.append((temp[1],temp[0],i))
        
            #sort delta_chk based on first value of tuple in descending order
            delta_chk = sorted(delta_chk,key = lambda x: x[0],reverse = True)

            top_delta = delta_chk[0]
            # print("Delta :", top_delta[0])
            if (top_delta[0] < 0):
                # print("convergence reached at round ",iter)
                break

            sum = sum - chunk_embedding[query_chunks[top_delta[1]]] + chunk_embedding[top_delta[2]]
            query_chunks[top_delta[1]] = top_delta[2]
            # print(query_chunks)

    return query_chunks
    


    

In [ ]:
filename="../FinalExp/Wikiqueries.txt"
fp = open(filename,"r")
querylist = fp.readlines()
fp.close()
print(querylist)
print(len(querylist))

In [ ]:
uq_chk_factor= 0.25
lt_factor=0.7
top_k = 7

print("Unique chunks factor ", uq_chk_factor)
print("Length factor ", lt_factor)
print("Top k chunks ", top_k)



for q in querylist:
        corelation = query_corelation(q)
        # tp1 = [(corelation[i],i) for i in range(len(corelation))]
        # #sort based on corelation
        # tp1 = sorted(tp1,key = lambda x: x[0],reverse = True)
        # print("Top K Scores: ")
        # for i in range(top_k):
        #     print(tp1[i][0],end =" ")
        # print("")

        # print("Top K Chunks: ")
        # for i in range(top_k):
        #     print(tp1[i][1],end =" ")
        # print("")
        print("-----------------")

        query_chunks = method2retreiver(chunk_lst,corelation,embed_lst,unique_chunks_factor=uq_chk_factor,length_factor=lt_factor,top_k=top_k)
        sum = 0
        for i in query_chunks:
            # print(len(chunk_lst[i]),end = " ")
            sum += len(chunk_lst[i])
        


        retrieved_ch_lst = [(query_chunks[i],corelation[query_chunks[i]],len(chunk_lst[query_chunks[i]])) for i in range(top_k)]
        retrieved_ch_lst = sorted(retrieved_ch_lst,key = lambda x: x[1],reverse = True)
        # print("")
        print("Scores: ")
        for i in range(top_k):
            print(retrieved_ch_lst[i][1],end =" ")
        print("")

        print("Chunks: ")
        for i in range(top_k):
            print(retrieved_ch_lst[i][0],end =" ")
        print("")

        print("Lengths: ")
        for i in range(top_k):
            print(retrieved_ch_lst[i][2],end =" ")
        print("")

        print("Sum of lengths: ")
        # print("-----------------")
        print(sum)
        # print(query_chunks)
        print("-----------------")


In [ ]:
# fp3 = open("recursive retreival.txt","w")
# for i in range(top_k):
#     fp3.write(f"chunk {indices[i]}: \n" + chunk[i] + "\n")
#     fp3.write("---------------------------\n\n\n")
# fp3.close()

In [ ]:
# print(chunk_metadata[6774])
fp = open("view_chunk.txt","w")
chunk_no = 4121
i1 = metadata_index1[chunk_no]
i2 = metadata_index2[chunk_no]
print(chunk_metadata[i1][i2])
print(len(chunk_lst[chunk_no]))
fp.write(chunk_lst[chunk_no] + "\n")
fp.close()

In [ ]:
##Ignore the below code , This is replaced by another API calls.
from llama_cpp import Llama



# Model_path = "/content/drive/MyDrive/Models/llama-2-7b.Q8_0.gguf"
Model_path = "/home/sasi/Desktop/Final_Sem/CS4805 BTP/models/claude2-alpaca-13b.Q4_K_M.gguf"


#execute the mutated query on llm

MODEL_Q2_K = Llama(
    model_path=Model_path,
    n_gpu_layers=-1,
    n_threads = 10,
    seed = 1432,
    n_ctx=2048)

In [ ]:


filename = '/home/sasi/Desktop/Final_Sem/CS4805 BTP/Exp1/LatestRussianelections/Alpaca-1_32.txt'
fp = open(filename,'a')
for i in range(0,len(querylist)):
  chunk_corelation = query_corelation(querylist[i])
  transformed_query = query_mutation(querylist[i],chunk_lst,chunk_corelation)
  print(transformed_query)
  fp.write("Q>" + transformed_query + "\n")
  fp.write(query_llm(MODEL_Q2_K, transformed_query ))
  fp.write("\n\n")
  fp.flush()


fp.close()
  
  # query_llm(MODEL_Q2_K, querylist[i] )


In [ ]:
filename = "/home/sasi/Desktop/Final_Sem/CS4805 BTP/Exp1/LatestRussianelections/Alpaca.txt"
fp = open(filename,'a')
for i in range(0,len(querylist)):
  fp.write("Q>" + querylist[i] + "\n" )
  fp.write(query_llm(MODEL_Q2_K, querylist[i] ) + "\n\n")
  fp.flush()

fp.close()
  
  

In [ ]:
qpfile="/home/sasi/Desktop/Final_Sem/CS4805_BTP/Exp2/entire_questions.txt"

qpfile = open(qpfile,'r')

queries = qpfile.readlines()

qpfile.close()

ragqueries="/home/sasi/Desktop/Final_Sem/CS4805_BTP/Exp2/entire_wiki_context.txt"

ragfile=open(ragqueries,'w')
for query in queries:
    q = query.replace('\n','')
    lis =chunking(q)
    # print(lis[1])
    # print(chunk_metadata[ lis[2][0] ] )
    ragfile.write(" ".join(lis[1]) + "\n")
    ragfile.flush()


ragfile.close()

# for i in range(k):
#     j = sorted_data[i]
#     print(chunk_corelation[j])
#     print(chunk_lst[j])



In [ ]:
query_llm(MODEL_Q2_K, transformed_query )

In [ ]:
query1="What is Constrained Causal Bayesian Optimization? Answer breifly."
query_llm(MODEL_Q2_K, query1 )

In [ ]:
print("Jagame maaya")